In [1]:
sample_n_list = [5_000, 10_000, 50_000, 100_000, 500_000, 1_000_000]
sample_n = sample_n_list[5]
print(sample_n)
language_option_list = ['english', 'italian', 'multi']
language_option = language_option_list[1]
print(language_option)

1000000
italian


In [2]:
# monolingual
# model_name = 'bert-base-uncased'
# model_name = 'roberta-base'

# multilingual
# model_name = 'bert-base-multilingual-cased'
# model_name = 'xlm-roberta-base'

# italian
# model_name = 'Musixmatch/umberto-commoncrawl-cased-v1'
model_name = 'm-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0'

In [3]:
import torch
from datasets import Dataset
from transformers import BertTokenizer, BertForMaskedLM, DataCollatorForLanguageModeling, AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments
import pickle
from typing import Dict

tokenizer = AutoTokenizer.from_pretrained(model_name)

# create datasets

In [4]:
# Load your own corpus
def load_custom_corpus(file_path):
    import pandas as pd
    df = pd.read_csv(file_path)
    df = df.fillna('')
    df = df[df['data_type'] == 'unknown'] # only take rows that do not belong to train/dev/test of IFD-EN-5203
    sentences = [sent for sent in df['text']]
    return {'text': sentences}

def tokenize_function(examples):
    return tokenizer((examples['text']), truncation=True, max_length=128, padding='max_length')

file_path_en = '/kaggle/input/dit-thesis-datasets/datasets_kaggle/English/Incels.is/IFC-22-EN_updated.csv'  # Replace this with the path to your corpus file
corpus_en = load_custom_corpus(file_path_en)
dataset_en = Dataset.from_dict(corpus_en)
tokenized_dataset_en = dataset_en.map(tokenize_function, batched=True, remove_columns=['text'])

file_path_it = '/kaggle/input/dit-thesis-datasets/datasets_kaggle/Italian/Il_forum_dei_brutti/IFC-22-IT_updated.csv'  # Replace this with the path to your corpus file
corpus_it = load_custom_corpus(file_path_it)
dataset_it = Dataset.from_dict(corpus_it)
tokenized_dataset_it = dataset_it.map(tokenize_function, batched=True, remove_columns=['text'])
print(tokenized_dataset_en)
print(tokenized_dataset_it)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: DtypeWarning: Columns (2,7) have mixed types.Specify dtype option on import or set low_memory=False.
  


  0%|          | 0/4753 [00:00<?, ?ba/s]

  0%|          | 0/628 [00:00<?, ?ba/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4752924
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 627839
})


# create pickles

In [5]:
# path_dataset_en = '/kaggle/working/'
# filename_pickle_en = 'IFC-22-EN_'+str(int(len(tokenized_dataset_en)/1000))+'k.pickle'
# path_pickle_en = os.path.join(path_dataset_en,filename_pickle_en)

# if not os.path.isfile(path_pickle_en):
#     # Save the tokenized_dataset as a pickle file
#     with open(path_pickle_en, 'wb') as file:
#         pickle.dump(tokenized_dataset_en, file)
# else:
#     print(f"{path_pickle_en} already exists. Not overwriting.")

# path_dataset_it = '/kaggle/working/'
# filename_pickle_it = 'IFC-22-IT_'+str(int(len(tokenized_dataset_it)/1000))+'k.pickle'
# path_pickle_it = os.path.join(path_dataset_it,filename_pickle_it)

# if not os.path.isfile(path_pickle_it):
#     # Save the tokenized_dataset as a pickle file
#     with open(path_pickle_it, 'wb') as file:
#         pickle.dump(tokenized_dataset_it, file)
# else:
#     print(f"{path_pickle_it} already exists. Not overwriting.")

In [6]:
# shuffle and sample datasets

# # Set a seed to ensure reproducibility when shuffling
# seed = 42
# sample_n_list = [5_000, 10_000, 50_000, 100_000, 500_000]

# for sample_n in sample_n_list:
#     shuffled_dataset_en = tokenized_dataset_en.shuffle(seed=seed)
#     sampled_dataset_en = shuffled_dataset_en.select(range(sample_n))
#     print(sampled_dataset_en)

#     shuffled_dataset_it = tokenized_dataset_it.shuffle(seed=seed)
#     sampled_dataset_it = shuffled_dataset_it.select(range(sample_n))
#     print(sampled_dataset_it)

#     path_dataset_en = '/home/pgajo/working/data/datasets/English/Incels.is'
#     filename_pickle_en = 'IFC-22-EN_'+str(int(sample_n/1000))+'k.pickle'
#     path_pickle_en = os.path.join(path_dataset_en,filename_pickle_en)

#     if not os.path.isfile(path_pickle_en):
#         # Save the tokenized_dataset as a pickle file
#         with open(path_pickle_en, 'wb') as file:
#             pickle.dump(sampled_dataset_en, file)
#     else:
#         print(f"{path_pickle_en} already exists. Not overwriting.")

#     path_dataset_it = '/home/pgajo/working/data/datasets/Italian/Il_forum_dei_brutti'
#     filename_pickle_it = 'IFC-22-IT_'+str(int(sample_n/1000))+'k.pickle'
#     path_pickle_it = os.path.join(path_dataset_it,filename_pickle_it)

#     if not os.path.isfile(path_pickle_it):
#         # Save the tokenized_dataset as a pickle file
#         with open(path_pickle_it, 'wb') as file:
#             pickle.dump(sampled_dataset_it, file)
#     else:
#         print(f"{path_pickle_it} already exists. Not overwriting.")

In [7]:
# # load pickles

# path_pickle_en = '/kaggle/input/dit-thesis-datasets/datasets_kaggle/English/Incels.is/IFC-22-EN_4752k.pickle'

# # Load the tokenized_dataset from the pickle file
# with open(path_pickle_en, 'rb') as file:
#     tokenized_dataset_en = pickle.load(file)

# path_pickle_it = '/kaggle/input/dit-thesis-datasets/datasets_kaggle/Italian/Il_forum_dei_brutti/IFC-22-IT_627k.pickle'

# # Load the tokenized_dataset from the pickle file
# with open(path_pickle_it, 'rb') as file:
#     tokenized_dataset_it = pickle.load(file)
    
# print(tokenized_dataset_en)
# print(tokenized_dataset_it)

In [8]:
# shuffle and sample datasets

# Set a seed to ensure reproducibility when shuffling
seed = 42

from datasets import concatenate_datasets

if language_option == 'multi':
    shuffled_dataset_en = tokenized_dataset_en.shuffle(seed=seed)
    sampled_dataset_en = shuffled_dataset_en.select(range(sample_n))
    print('sampled_dataset_en\n',sampled_dataset_en)

    shuffled_dataset_it = tokenized_dataset_it.shuffle(seed=seed)
    sampled_dataset_it = shuffled_dataset_it.select(range(sample_n))
    print('sampled_dataset_it\n', sampled_dataset_it)

    # Assuming you have loaded the two datasets as `dataset1` and `dataset2`
    merged_dataset = concatenate_datasets([sampled_dataset_en, sampled_dataset_it])
    train_dataset = merged_dataset.shuffle(seed=seed)
    print('train_dataset_multi\n', train_dataset)
    
if language_option == 'english':
    
    shuffled_dataset_en = tokenized_dataset_en.shuffle(seed=seed)
    train_dataset = shuffled_dataset_en.select(range(sample_n))
    print('train_dataset_en\n', train_dataset)
    
if language_option == 'italian':
    
    shuffled_dataset_it = tokenized_dataset_it.shuffle(seed=seed)
    if sample_n > len(shuffled_dataset_it):
        sample_n = len(shuffled_dataset_it)
    train_dataset = shuffled_dataset_it.select(range(sample_n))
    print('train_dataset_it\n', train_dataset)
    print('shuffled_dataset_it:',shuffled_dataset_it)

train_dataset_it
 Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 627839
})
shuffled_dataset_it: Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 627839
})


define save path

In [9]:
# Replace these with the appropriate model and tokenizer names
new_model_name = 'incel-'+model_name.split('/')[-1]+'-'+str(int(len(train_dataset)/1000))+'k_'+language_option

# Save the model and tokenizer to a directory
output_dir = "/kaggle/working/"

import os
# Create the directory if it doesn't exist
model_path = os.path.join(output_dir,new_model_name)

if not os.path.exists(model_path):
    os.makedirs(model_path)
print('####################################')
print(model_path)
print('####################################')

####################################
/kaggle/working/incel-bert_uncased_L-12_H-768_A-12_italian_alb3rt0-627k_italian
####################################


In [10]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

model = AutoModelForMaskedLM.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir='/kaggle/working/results/',
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='/kaggle/working/logs/',
    logging_steps=100,
    report_to='none',
    disable_tqdm=False,
)

# extend the huggingface Trainer class to make custom methods
class CustomTrainer(Trainer):
    def log(self, logs: Dict[str, float]):
        # Call the original `log` method to preserve its functionality
        super().log(logs)

        # Calculate total steps
        total_steps = len(self.train_dataset) * self.args.num_train_epochs // self.args.per_device_train_batch_size
        if self.args.world_size > 1:
            total_steps = total_steps // self.args.world_size

        # Calculate the percentage of completed steps
        progress_percentage = 100 * self.state.global_step / total_steps

        # Print the custom message
        print("Global step:", self.state.global_step)
        print(f"MOM LOOK I'M LOGGING: {progress_percentage:.2f}% steps completed ({self.state.global_step}/{total_steps})")

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


trainer.train(
    # resume_from_checkpoint = True
    )

Some weights of the model checkpoint at m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running 

Step,Training Loss
100,3.565900
200,3.424900
300,3.443000
400,3.379400
500,3.326200
600,3.297500
700,3.283800
800,3.344400
900,3.276000
1000,3.249800


Global step: 100
MOM LOOK I'M LOGGING: 0.51% steps completed (100/19619)
Global step: 200
MOM LOOK I'M LOGGING: 1.02% steps completed (200/19619)
Global step: 300
MOM LOOK I'M LOGGING: 1.53% steps completed (300/19619)
Global step: 400
MOM LOOK I'M LOGGING: 2.04% steps completed (400/19619)
Global step: 500
MOM LOOK I'M LOGGING: 2.55% steps completed (500/19619)
Global step: 600
MOM LOOK I'M LOGGING: 3.06% steps completed (600/19619)
Global step: 700
MOM LOOK I'M LOGGING: 3.57% steps completed (700/19619)
Global step: 800
MOM LOOK I'M LOGGING: 4.08% steps completed (800/19619)
Global step: 900
MOM LOOK I'M LOGGING: 4.59% steps completed (900/19619)
Global step: 1000
MOM LOOK I'M LOGGING: 5.10% steps completed (1000/19619)
Global step: 1100
MOM LOOK I'M LOGGING: 5.61% steps completed (1100/19619)
Global step: 1200
MOM LOOK I'M LOGGING: 6.12% steps completed (1200/19619)
Global step: 1300
MOM LOOK I'M LOGGING: 6.63% steps completed (1300/19619)
Global step: 1400
MOM LOOK I'M LOGGING: 7.1

Saving model checkpoint to /kaggle/working/results/checkpoint-10000
Configuration saved in /kaggle/working/results/checkpoint-10000/config.json
Configuration saved in /kaggle/working/results/checkpoint-10000/generation_config.json


Global step: 10000
MOM LOOK I'M LOGGING: 50.97% steps completed (10000/19619)


Model weights saved in /kaggle/working/results/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in /kaggle/working/results/checkpoint-10000/tokenizer_config.json
Special tokens file saved in /kaggle/working/results/checkpoint-10000/special_tokens_map.json


Global step: 10100
MOM LOOK I'M LOGGING: 51.48% steps completed (10100/19619)
Global step: 10200
MOM LOOK I'M LOGGING: 51.99% steps completed (10200/19619)
Global step: 10300
MOM LOOK I'M LOGGING: 52.50% steps completed (10300/19619)
Global step: 10400
MOM LOOK I'M LOGGING: 53.01% steps completed (10400/19619)
Global step: 10500
MOM LOOK I'M LOGGING: 53.52% steps completed (10500/19619)
Global step: 10600
MOM LOOK I'M LOGGING: 54.03% steps completed (10600/19619)
Global step: 10700
MOM LOOK I'M LOGGING: 54.54% steps completed (10700/19619)
Global step: 10800
MOM LOOK I'M LOGGING: 55.05% steps completed (10800/19619)
Global step: 10900
MOM LOOK I'M LOGGING: 55.56% steps completed (10900/19619)
Global step: 11000
MOM LOOK I'M LOGGING: 56.07% steps completed (11000/19619)
Global step: 11100
MOM LOOK I'M LOGGING: 56.58% steps completed (11100/19619)
Global step: 11200
MOM LOOK I'M LOGGING: 57.09% steps completed (11200/19619)
Global step: 11300
MOM LOOK I'M LOGGING: 57.60% steps completed 



Training completed. Do not forget to share your model on huggingface.co/models =)




Global step: 19620
MOM LOOK I'M LOGGING: 100.01% steps completed (19620/19619)


TrainOutput(global_step=19620, training_loss=2.9302401077006084, metrics={'train_runtime': 14688.2101, 'train_samples_per_second': 42.744, 'train_steps_per_second': 1.336, 'total_flos': 4.135956359223706e+16, 'train_loss': 2.9302401077006084, 'epoch': 1.0})

In [11]:
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in /kaggle/working/incel-bert_uncased_L-12_H-768_A-12_italian_alb3rt0-627k_italian/config.json
Configuration saved in /kaggle/working/incel-bert_uncased_L-12_H-768_A-12_italian_alb3rt0-627k_italian/generation_config.json
Model weights saved in /kaggle/working/incel-bert_uncased_L-12_H-768_A-12_italian_alb3rt0-627k_italian/pytorch_model.bin
tokenizer config file saved in /kaggle/working/incel-bert_uncased_L-12_H-768_A-12_italian_alb3rt0-627k_italian/tokenizer_config.json
Special tokens file saved in /kaggle/working/incel-bert_uncased_L-12_H-768_A-12_italian_alb3rt0-627k_italian/special_tokens_map.json


('/kaggle/working/incel-bert_uncased_L-12_H-768_A-12_italian_alb3rt0-627k_italian/tokenizer_config.json',
 '/kaggle/working/incel-bert_uncased_L-12_H-768_A-12_italian_alb3rt0-627k_italian/special_tokens_map.json',
 '/kaggle/working/incel-bert_uncased_L-12_H-768_A-12_italian_alb3rt0-627k_italian/vocab.txt',
 '/kaggle/working/incel-bert_uncased_L-12_H-768_A-12_italian_alb3rt0-627k_italian/added_tokens.json',
 '/kaggle/working/incel-bert_uncased_L-12_H-768_A-12_italian_alb3rt0-627k_italian/tokenizer.json')